In [4]:
from Mollification import *

import trimesh
import os
import time

import meshio as mio

from buildGlueMap import *

root_folder = os.getcwd()

In [5]:
# read paths from a text file, each line is a path
paths = []
# make sure encoding = "utf-16 le" for the tooNiceMeshPaths.txt file


with open("filtering-thing10k/veryBadMeshPaths.txt", "r", encoding="utf-16") as f:
    for line in f:
        paths.append(line.strip())

# with open("filtering-thing10k/validMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())
# with open("filtering-thing10k/tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())
# with open("filtering-thing10k/large_tooNiceMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())
# with open("filtering-thing10k/large_kindaBadMeshPaths.txt", "r", encoding="utf-16") as f:
#     for line in f:
#         paths.append(line.strip())

# randomly order paths
import random
random.seed(42)
random.shuffle(paths)


print("Number of paths: ", len(paths))


Number of paths:  913


In [6]:
delta = [1e-4] #, 1e-3, 1e-2, 1e-1]

def func(FL, G, delta, i):
    if i == 0:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
    elif i == 1:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
    elif i == 2:
        return IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
    elif i == 3:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
    elif i == 4:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
    elif i == 5:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_MANHATTAN)
    elif i == 6:
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_EUCLIDEAN)
    elif i == 7:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 8:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 9:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 10:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
    elif i == 11:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 12:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 13:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MAX)
    elif i == 14:        
        return IntrinsicMollificationFL(FL, G, delta,  scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MAX)



time_glue = []

moll_labels = [
    "Constant Epsilon",

    "One by One Step (Local)",
    "One by One Interpolated (Local)",
    "Local Least Mollification Manhattan (Local)",
    "Local Least Mollification Euclidean (Local)",

    "Global Least Mollification Manhattan",
    "Global Least Mollification Euclidean",

    "One by One Step (Global, Mean)",
    "One by One Interpolated (Global, Mean)",
    "Local Least Mollification Manhattan (Global, Mean)",
    "Local Least Mollification Euclidean (Global, Mean)",

    "One by One Step (Global, Max)",
    "One by One Interpolated (Global, Max)",
    "Local Least Mollification Manhattan (Global, Max)",
    "Local Least Mollification Euclidean (Global, Max)"
]

times = np.zeros((len(moll_labels), len(delta)))

n_moll = np.zeros((len(moll_labels), len(delta)))

n_iter = np.zeros((len(moll_labels), len(delta)))

avg_moll = np.zeros((len(moll_labels), len(delta)))

avg_moll_for_mollified_edges = np.zeros((len(moll_labels), len(delta)))



for i in range(len(paths)):
    mesh = mio.read("../../GC/" + paths[i])
    F = mesh.cells_dict["triangle"]
    V = mesh.points
    # [V,F]= igl.read_triangle_mesh(os.path.join(
                # root_folder, "../../GC/", paths[i]))
    print("Mesh ", i + 1, " out of ", len(paths), ". Number of vertices: ", len(V), ". Number of faces: ", len(F), ".")


    start = time.time()
    G = build_gluing_map(F)
    end = time.time()
    time_glue.append(end - start)

    FL0 = igl.edge_lengths(V, F)

    avg_edge_length = np.mean(FL0)

    for j in range(len(delta)):
        for k in range(len(moll_labels)):
            if k > 6:
                continue
            start = time.time()
            FL, nMoll, nIter = func(FL0, G, delta[j], k)
            end = time.time()
            times[k][j] += end - start
            n_moll[k][j] += nMoll / len(FL0 * 3)
            n_iter[k][j] += nIter
            avg_moll[k][j] += np.sum(FL - FL0) / np.sum(FL0)
            if(nMoll > 0):
                avg_moll_for_mollified_edges[k][j] += np.sum(FL - FL0) / (avg_edge_length * nMoll * 3)
            print(i + 1, j, k)


    if((i + 1) % 10 == 0 and i != 0):
        print("Finished ", i + 1, " meshes.")
        print("Average time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
        print("_________________________________________________________")

        for j in range(len(delta)):
            for k in range(len(moll_labels)):
                if k > 6:
                    continue
                print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
                print("Average time: ", times[k][j] / (i + 1))
                print("Average number of mollified edges: ", n_moll[k][j] / (i + 1))
                print("Average number of iterations: ", n_iter[k][j] / (i + 1))
                print("Average mollification: ", avg_moll[k][j] / (i + 1))
                print("Average mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / (i + 1))
                print("_________________________________________________________")


print("Finished All meshes.")
print("Average time to glue: ", np.mean(time_glue), " Std dev time to glue: ", np.std(time_glue))
print("_________________________________________________________")
for j in range(len(delta)):
    for k in range(len(moll_labels)):
        print("Delta: ", delta[j], " Mollification: ", moll_labels[k])
        print("Average time: ", times[k][j] / len(paths))
        print("Average number of mollified edges: ", n_moll[k][j] / len(paths))
        print("Average number of iterations: ", n_iter[k][j] / len(paths))
        print("Average mollification: ", avg_moll[k][j] / len(paths))
        print("Average mollification for mollified edges: ", avg_moll_for_mollified_edges[k][j] / len(paths))
        print("_________________________________________________________")

Mesh  1  out of  913 . Number of vertices:  28 . Number of faces:  52 .
0.0 3.823684778905178e-06
1 0 0
0.0 3.823684778905178e-06
1 0 1
-0.0001678317533336804 3.823684778905178e-06
1 0 2
0.0 3.823684778905178e-06
1 0 3
-0.00014466833531301404 3.823684778905178e-06
1 0 4
0.0 3.823684778905178e-06
1 0 5
-1.3223735528811176e-07 3.823684778905178e-06
1 0 6
Mesh  2  out of  913 . Number of vertices:  488 . Number of faces:  972 .
0.0 8.4840016214798e-06
2 0 0
0.0 8.4840016214798e-06
2 0 1
-7.062365557919392e-06 8.4840016214798e-06
2 0 2
0.0 8.4840016214798e-06
2 0 3
-9.74395832464836e-07 8.4840016214798e-06
2 0 4
0.0 8.4840016214798e-06
2 0 5
-2.229322504376796e-09 8.4840016214798e-06
2 0 6
Mesh  3  out of  913 . Number of vertices:  3682 . Number of faces:  7384 .
0.0 1.929522335529327e-06
3 0 0
1.1920929e-07 1.929522335529327e-06
3 0 1
0.0 1.929522335529327e-06
3 0 2
3.054738e-07 1.929522335529327e-06
3 0 3
-3.5761332e-07 1.929522335529327e-06
3 0 4
3.054738e-07 1.929522335529327e-06
3 0 

AssertionError: 

In [ ]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            # root_folder, "../../GC/TestEdgeLengths", "11.stl"))
            # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "36069.stl"))
            # root_folder, "../../GC/Thingi10K/Thingi10K/raw_meshes", "93743.stl"))
            # root_folder, "../data", "goathead.obj"))
            root_folder, "../data", "dime_counting_tube.stl"))


In [ ]:
mesh = mio.read("../data/dime_counting_tube.stl")
F = mesh.cells_dict["triangle"]
V = mesh.points

In [ ]:
delta = 1e-2
FL = igl.edge_lengths(V, F)

In [ ]:
# time before
end0 = time.time()
FL1, nMoll1, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end1 = time.time()
print("Constant Epsilon took: ", end1 - end0)

FL2, nMoll2, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP)
end2 = time.time()
print("One by one step took: ", end2 - end1)

FL3, nMoll3, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)
end3 = time.time()
print("One by one interpolated took: ", end3 - end2)

FL4, nMoll4, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN)
end4 = time.time()
print("Local least mollification Manhattan took: ", end4 - end3)

FL5, nMoll5, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN)
end5 = time.time()
print("Local least mollification Euclidean took: ", end5 - end4)

[0 1 0 0]
[0 2 0 2] >
[0 2 0 2]
[1 2 0 1] >
[1 2 0 1]
[3 4 1 0] >
[3 4 1 0]
[3 5 1 2] >
[3 5 1 2]
[4 5 1 1] >
[4 5 1 1]
[6 7 2 0] >
[6 7 2 0]
[6 8 2 2] >
[6 8 2 2]
[7 8 2 1] >
[7 8 2 1]
[ 9 10  3  0] >
[ 9 10  3  0]
[ 9 11  3  2] >
[ 9 11  3  2]
[10 11  3  1] >
[10 11  3  1]
[12 13  4  0] >
[12 13  4  0]
[12 14  4  2] >
[12 14  4  2]
[13 14  4  1] >
[13 14  4  1]
[15 16  5  0] >
[15 16  5  0]
[15 17  5  2] >
[15 17  5  2]
[16 17  5  1] >
[16 17  5  1]
[18 19  6  0] >
[18 19  6  0]
[18 20  6  2] >
[18 20  6  2]
[19 20  6  1] >
[19 20  6  1]
[21 22  7  0] >
[21 22  7  0]
[21 23  7  2] >
[21 23  7  2]
[22 23  7  1] >
[22 23  7  1]
[24 25  8  0] >
[24 25  8  0]
[24 26  8  2] >
[24 26  8  2]
[25 26  8  1] >
[25 26  8  1]
[27 28  9  0] >
[27 28  9  0]
[27 29  9  2] >
[27 29  9  2]
[28 29  9  1] >
[28 29  9  1]
[30 31 10  0] >
[30 31 10  0]
[30 32 10  2] >
[30 32 10  2]
[31 32 10  1] >
[31 32 10  1]
[33 34 11  0] >
[33 34 11  0]
[33 35 11  2] >
[33 35 11  2]
[34 35 11  1] >
[34 35 11  1]
[36 

IndexError: index 480 is out of bounds for axis 0 with size 480

In [ ]:
print(G)

In [ ]:
G = build_gluing_map(F)


end5 = time.time()

FL6, nMoll6, nIter6 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_STEP, pooling=MOLLIFICATION_POOLING.MEAN)
end6 = time.time()
print("Sequential global, one by one step, mean pooling took: ", end6 - end5)

 FL7, nMoll7, nIter7 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED, pooling=MOLLIFICATION_POOLING.MEAN)
 end7 = time.time()
 print("Sequential global, one by one interpolated, mean pooling took: ", end7 - end6)
 FL8, nMoll8, nIter8 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_MANHATTAN, pooling=MOLLIFICATION_POOLING.MEAN)
 end8 = time.time()
 print("Sequential global, local least mollification Manhattan, mean pooling took: ", end8 - end7)
 FL9, nMoll9, nIter9 = IntrinsicMollificationFL(L, G, delta, scheme=MOLLIFICATION_SCHEME.SEQUENTIAL_GLOBAL, local_scheme=MOLLIFICATION_LOCAL_SCHEME.LOCAL_LEAST_MOLLIFICATION_EUCLIDEAN, pooling=MOLLIFICATION_POOLING.MEAN)
 end9 = time.time()
 print("Sequential global, local least mollification Euclidean, mean pooling took: ", end9 - end8)

1 27 27
2 108 135
3 33 168
4 36 204
5 36 240
6 35 275
7 35 310
8 35 345
9 35 380
10 35 415
11 35 450
12 35 485
13 35 520
14 35 555
15 35 590
16 35 625
17 35 660
18 34 694
19 34 728
20 34 762
21 33 795
22 33 828
23 33 861
24 30 891
25 25 916
26 22 938
27 11 949
28 6 955
29 3 958
30 1 959
31 0 959
Sequential global, one by one step, mean pooling took:  0.058502912521362305


In [ ]:
print("Constant Epsilon nMoll", nMoll1)
print("One by one step nMoll", nMoll2)
print("One by one interpolated nMoll", nMoll3)
print("Local least mollification Manhattan", nMoll4)
print("Local least mollification Euclidean", nMoll5) 
print("Sequential global, one by one step, mean pooling", nMoll6, "with", nIter6, "iterations")
print("Sequential global, one by one interpolated, mean pooling", nMoll7, "with", nIter7, "iterations")
print("Sequential global, local least mollification Manhattan, mean pooling", nMoll8, "with", nIter8, "iterations")
print("Sequential global, local least mollification Euclidean, mean pooling", nMoll9, "with", nIter9, "iterations")

Constant Epsilon nMoll 699466
One by one step nMoll 621
One by one interpolated nMoll 621
Local least mollification Manhattan 621
Local least mollification Euclidean 621
Sequential global, one by one step, mean pooling 621 with 2 iterations
Sequential global, one by one interpolated, mean pooling 621 with 2 iterations
Sequential global, local least mollification Manhattan, mean pooling 621 with 2 iterations
Sequential global, local least mollification Euclidean, mean pooling 621 with 2 iterations


In [ ]:
print(np.sum(FL1 - FL0) / np.sum(FL0))
print(np.sum(FL2 - FL0) / np.sum(FL0))
print(np.sum(FL3 - FL0) / np.sum(FL0))
print(np.sum(FL4 - FL0) / np.sum(FL0))
print(np.sum(FL5 - FL0) / np.sum(FL0))
print(np.sum(FL6 - FL0) / np.sum(FL0))
print(np.sum(FL7 - FL0) / np.sum(FL0))
print(np.sum(FL8 - FL0) / np.sum(FL0))
print(np.sum(FL9 - FL0) / np.sum(FL0))

0.009999969844763803
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06
1.9515460139100948e-06
3.474920234048739e-06
1.9443048566123697e-06
1.9480259108943147e-06


In [ ]:
F = np.array([F]).repeat(2, axis=0)
print(F.shape)

(2, 144046, 3)


In [ ]:
b = np.array([- delta,   - delta,    - delta,    0,     0,    0])
print(b, b[3])

[-0.1 -0.1 -0.1  0.   0.   0. ] 0.0


In [ ]:

print(times)
print(times[0][0])

[[[0.008999824523925781, 0.009971857070922852, 0.0074045658111572266, 0.007978677749633789, 0.007978200912475586, 0.007978677749633789, 0.006981372833251953, 0.0069806575775146484, 0.007978200912475586, 0.00698399543762207, 0.0020787715911865234, 0.01875782012939453, 0.012966156005859375, 0.20042777061462402, 0.21538996696472168, 0.22843718528747559, 0.24833154678344727, 0.22535395622253418, 0.23342514038085938, 0.2054445743560791, 0.22211956977844238, 0.22598624229431152, 0.21291351318359375, 0.22838973999023438, 0.2183852195739746, 0.2104785442352295, 0.0209505558013916, 0.0209810733795166, 0.02093958854675293, 0.02194070816040039, 0.02194833755493164, 0.020937442779541016, 0.02293872833251953, 0.024903535842895508, 0.02397322654724121, 0.023938894271850586, 0.020876646041870117, 0.025930166244506836, 0.025974512100219727, 0.042885780334472656, 0.04487967491149902, 0.044912099838256836, 0.04484891891479492, 0.05186605453491211, 0.04487323760986328, 0.040891408920288086, 0.04391884803